In [6]:
import pandas as pd
import re
from datetime import datetime

# === 1. 读取原始文件 ===
# 请把路径改成你本地文件的实际位置，例如：
# file_path = "C:/Users/YourName/Downloads/melbourne_cbd_bars.csv"
file_path = "restaurant_english_clean_desc.csv"
df = pd.read_csv(file_path)

def to_24h_auto(time_str, reference=None):
    """自动推断并转成24小时制"""
    time_str = time_str.strip().lower().replace(" ", "").replace(" ", "")
    try:
        # 有 am/pm 直接解析
        if re.search(r'(am|pm)', time_str):
            return datetime.strptime(time_str, "%I:%M %p").strftime("%H:%M")
        # 无 am/pm 的情况
        h, m = (time_str.split(":") + ["00"])[:2]
        h, m = int(h), int(m)
        # 若参考时间是下午的，就自动补 PM
        if reference and re.search("pm", reference.lower()):
            h = (h + 12) if h < 12 else h
        return f"{h:02d}:{m:02d}"
    except:
        return time_str

def normalize_opening_hours_v3(text):
    """增强版：支持无 am/pm 时间自动推断"""
    if pd.isna(text):
        return text
    text = text.replace("–", "-").replace("—", "-")
    days = text.split("|")
    new_days = []

    for day in days:
        day = day.strip()
        if "closed" in day.lower():
            new_days.append(day)
            continue

        match = re.findall(r"(\d{1,2}(:\d{2})?)\s*-\s*(\d{1,2}(:\d{2})?)", day)
        if not match:
            new_days.append(day)
            continue

        start_raw = match[0][0]
        end_raw = match[0][2]

        # 自动判断 AM/PM 缺失的情况
        if re.search(r'(am|pm)', start_raw, re.IGNORECASE) or re.search(r'(am|pm)', end_raw, re.IGNORECASE):
            start_24 = to_24h_auto(start_raw, reference=end_raw)
            end_24 = to_24h_auto(end_raw, reference=start_raw)
        else:
            # 都没写 AM/PM，智能推断
            start_h = int(start_raw.split(":")[0])
            end_h = int(end_raw.split(":")[0])
            if end_h < start_h:  # 跨天
                start_24, end_24 = f"{start_h:02d}:00", f"{end_h:02d}:00"
            else:
                start_24, end_24 = f"{start_h:02d}:00", f"{end_h:02d}:00"

        new_day = re.sub(r"(\d{1,2}(:\d{2})?)\s*-\s*(\d{1,2}(:\d{2})?)",
                         f"{start_24}-{end_24}", day)
        new_days.append(new_day)

    return " | ".join(new_days)

# 应用新函数
df["openinghour"] = df["openinghour"].apply(normalize_opening_hours_v3)

# === 4. 保存新文件 ===
output_path = "restau.csv"
df.to_csv(output_path, index=False)

print("✅ 转换完成！文件已保存为：", output_path)



✅ 转换完成！文件已保存为： restau.csv


In [15]:
import pandas as pd
import re

# ===== 配置 =====
INPUT_FILE = "restaurant_english_clean_desc.csv"   # 输入文件
OUTPUT_FILE = "restaurant_openhour_cleaned.csv"    # 输出文件
OPENING_COL = "openinghour"                        # 含营业时间的列名

def normalize_opening_text(text: str):
    """统一符号、去掉引号"""
    if pd.isna(text) or not isinstance(text, str):
        return None
    text = text.strip().strip('"').strip("'")
    text = text.replace(" ", " ").replace(" ", " ").replace("–", "-")
    text = re.sub(r"\s+", " ", text)
    return text

def insert_missing_ampm(text: str):
    """
    自动补全缺失的 am/pm 到起始时间：
    例如：
      6:00 - 10:00 PM  → 6:00 PM - 10:00 PM
      6-10pm           → 6 PM - 10 PM
      6:00–10:00PM     → 6:00 PM - 10:00 PM
    """
    # ✅ 补全起始时间缺失的 am/pm
    text = re.sub(
        r"(\d{1,2}(?::\d{2})?)\s*[-–]\s*(\d{1,2}(?::\d{2})?)\s*(am|pm)\b",
        lambda m: f"{m.group(1)} {m.group(3).upper()} - {m.group(2)} {m.group(3).upper()}",
        text,
        flags=re.IGNORECASE,
    )

    # ✅ 标准化写法："6pm - 10pm" → "6 PM - 10 PM"
    text = re.sub(r"(\d)(am|pm)", lambda m: f"{m.group(1)} {m.group(2).upper()}", text, flags=re.IGNORECASE)

    return text

def clean_opening_hours(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return None

    text = normalize_opening_text(text)
    text = insert_missing_ampm(text)

    # 删除多余逗号前后空格
    text = re.sub(r"\s*,\s*", ", ", text)

    # 检查是否有任何 weekday 数据
    if not re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)", text, re.IGNORECASE):
        return None

    return text.strip()

# ===== 主程序 =====
if __name__ == "__main__":
    df = pd.read_csv(INPUT_FILE)

    # 自动检测营业时间列
    if OPENING_COL not in df.columns:
        candidates = [c for c in df.columns if 'hour' in c.lower() or 'open' in c.lower()]
        if candidates:
            OPENING_COL = candidates[0]
        else:
            raise ValueError("❌ 无法检测到营业时间列，请检查列名")

    print(f"✅ 检测到营业时间列：{OPENING_COL}")

    # 应用清洗函数
    df["openinghour_clean"] = df[OPENING_COL].apply(clean_opening_hours)

    # 删除空行
    df_clean = df.dropna(subset=["openinghour_clean"]).reset_index(drop=True)
    df_clean.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

    print(f"🎉 清洗完成！已保存至：{OUTPUT_FILE}")









✅ 检测到营业时间列：openinghour
🎉 清洗完成！已保存至：restaurant_openhour_cleaned.csv
